# Лабораторная работа №3
## Выполнил студент группы БСТ1901 Касенов Руслан 
### Задание №1
1 Реализовать методы поиска подстроки в строке. 
2 Добавить возможность ввода строки и подстроки с клавиатуры. 
3 Предусмотреть возможность существования пробела. 
4 Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. 
5 Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной
функции поиска, используемой в выбранном языке программирования. 
### Алгоритм Кнута-Морриса-Пратта

In [2]:
def prefix(subStr):
    prefFunc = [0]*len(subStr)
    for i in range(1,len(subStr)):
        k = prefFunc[i-1]
        while k > 0 and subStr[k] != subStr[i]:
            k = prefFunc[k-1]
        if subStr[k] == subStr[i]:
            k = k + 1
        prefFunc[i] = k
    return prefFunc

In [38]:
def kmp_search(subStr,line,case,space):
    space = space.lower()
    case = case.lower()
    _subStr = subStr
    _line = line
    if case == "no":
        subStr = _subStr.lower()
        line = _line.lower()
    if space =="no":
        _subStr = _subStr.replace(" ","")
        _line = _line.replace(" ","")
    index = -1
    f = prefix(subStr)
    k = 0
    for i in range(len(line)):
        while k > 0 and subStr[k] != line[i]:
            k = f[k-1]
        if subStr[k] == line[i]:
            k = k + 1
        if k == len(subStr):
            index = i - len(subStr) + 1
            break
    if space =="no":
            m=0
            n=0
            spacesNumber = 0
            while m<=k+1 and n<=len(line):
                if _line[m]==line[n]:
                    m+=1
                    n+=1
                else:
                    if line[n]==" ":
                        spacesNumber +=1
                        n+=1
            return index+spacesNumber
        else: 
            return index

In [51]:
line = input("input the string: ")
subStr = input("input the substring: ")
case = input("Case sensitivity, write Yes or No: ")
space = input("Space sensitivity, write Yes or No: ")
kmp_search(subStr,line,case,space)

input the string: abcabd
input the substring: cab
Case sensitivity, write Yes or No: No


2

### Упрощенный алгоритм Бойера-Мура


In [42]:
def displacement(subStr):
    table = [len(subStr)]*256
    for i in range(len(subStr) - 1):
        table[ord(subStr[i])] = len(subStr) - 1 - i
    return table

In [86]:
def bm_search(subStr, line,case,space):
    space = space.lower()
    case = case.lower()
    _subStr = subStr
    _line = line
    if case == "no":
        _subStr = _subStr.lower()
        _line = _line.lower()
    if space =="no":
        _subStr = _subStr.replace(" ","")
        _line = _line.replace(" ","")
        
    table = displacement(_subStr)
    i = len(_subStr) - 1
    j = i
    k = i
    while j >= 0 and i <= len(_line) - 1:
        j = len(_subStr) - 1
        k = i
        while j >= 0 and _line[k] == _subStr[j]:
            k-=1
            j-=1
        i += table[ord(_line[i])]
    
    if k >= len(_line) - len(_subStr):
        return -1
    else :
        if space =="no":
            m=0
            n=0
            spacesNumber = 0
            while m<=k+1 and n<=len(line):
                if _line[m]==line[n]:
                    m+=1
                    n+=1
                else:
                    if line[n]==" ":
                        spacesNumber +=1
                        n+=1
            return k+1+spacesNumber
        else: 
            return k+1




In [85]:
line = input("input the string: ")
subStr = input("input the substring: ")
case = input("Case sensitivity, write Yes or No: ")
space = input("Space sensitivity, write Yes or No: ")
bm_search(subStr,line,case,space)

input the string: aaab c d 
input the substring: bcd
Case sensitivity, write Yes or No: No
Space sensitivity, write Yes or No: No
aaabcd
3


3

### Задание №2

In [180]:
class Position:
    def __init__(self, position, start_distance,finish_distance):
        self.position = position
        self.start_distance = start_distance
        self.finish_distance = finish_distance

    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

    # Переопределяем метод less then для работы PriorityQueue
    def __lt__(self, other):
        return self.start_distance+self.finish_distance < other.start_distance+other.finish_distance

In [181]:
from queue import PriorityQueue

N = 4
# Генератор сдвигов
def shifts(position):
# Находим индекс нуля
    zeroPosition = position.index(0)
# Находим его позицию в поле 4х4, где  i это номер строки,а j номер столбца
    i, j = divmod(zeroPosition, N)    
    displacement = []
# В зависимости от местоположения нуля, смотрим варианты, куда его можно сдвинуть
    if i > 0: displacement.append(-N)     # вверх
    if i < N - 1: displacement.append(N)  # вниз
    if j > 0: displacement.append(-1)     # влево
    if j < N - 1: displacement.append(1)  # вправо
    for offset in displacement:
# считаем индексы новых позиций нуля
        swap = zeroPosition + offset
# Выводим новое состояние поля, где 0 сдвинут в одном из возможных направлений
        yield tuple(position[swap] if x==zeroPosition else position[zeroPosition] if x==swap else position[x] for x in range(N*N))

# Проверяет четность неправильных пар
def parityOfPairs(state):
    countOfPairs = 0
    for i in range(len(state)-1):
        if state[i] > state[i+1]:
            countOfPairs +=1
    return countOfPairs % 2

In [182]:
def fifteenGame(startState):
    terminalState = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
    # Если количество неправильных пар четное, то такая позиция нерешаема
    if parityOfPairs(startState)==0:
        print("Нет решений")
    else:
    # В противном случае создаем из начальной позиции кортеж
        startState= tuple(startState)
    # создаем экзмепляр класса, с кортежем стартовой позиции в качестве параметра,длиной пути от начала до текущей точки 
    # и длиной от текущей позиции до конца
        p = Position(startState, 0,0)
    # Выводим начальную позицию
        print(p)
        print()
    # Создаем экземпляр класса приоритетной очереди
        fieldStates= PriorityQueue()
    # Заносим в нее пару-это кортеж стартовой позиции, и вес данной вершины
        fieldStates.put(p)
    # Создаем множество посещенных вершин
        closePoints = set([p])
    # Создаем словарь в котором будем хранить позиции,где для каждой будет определена ее предыдущая позиция
        parents = {p.position: None}

    # Пока позиция не будет равна решению    
        while p.position != terminalState:
    # Получаем приоритетную позицию
            p =fieldStates.get()

    # для каждого варианта передвижения нуля в одном состоянии поля
            for k in shifts(p.position):
                count= 0
    # Если такой вариант передвижения не находится в списке посещенных
                if k not in closePoints:
    # Расчитаем растояние до терминального состояния, это количество цифр стоящих не на своих местах
                    for m in range(len(k)):
                        if k[m] != terminalState[m]:
                            count+=1

    # То мы заносим его в очередь, с состоянием в качестве параметра, длиной пути от старта до текущего сосотояния
    # и длиной до финиша
                    fieldStates.put(Position(k, p.start_distance +1,p.finish_distance+count))
    # в словарь добавляем новый ключ, то есть эту позицию, его значением будет предыдущая позиция
                    parents[k] = p
    # Добавляем этот вариант перемещения в посещенный
                    closePoints.add(k)


        path = []
        x = p
        previous = p
        while p.position != startState:
    # Берем из словаря родителя текущего состояния поля
            p = parents[p.position]
    # Запоминаем индекс нуля из текущего состояния и по этому индексу находим элемент, который сдвинули в предыдущем состоянии
            number = p.position[previous.position.index(0)]
            path.append(number)
    # Предыдущее состояние делаем текущим
            previous = p
    # Разворачиваем путь, чтобы получить путь от первого до последнего сдвига
        path.reverse()

        print(path)
        print(x)

In [184]:
startState = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9,11,12,10,14,15,0]
fifteenGame(startState)

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0

[15, 14, 10, 13, 9, 10, 14, 15]
  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14 15  0


# Вывод 
Я изучил основные алгоритмы поиска подстроки в строке. А так же реализовал алгоритм решения игры "Пятнашки"